# Projet numérique : Calcul Différentiel - Lignes de niveau

*Alexandre Thealler et Camille Srecki*

On cherche à développer un programme python permettant de calculer les __lignes de niveau__ d'une fontion $f$ de deux variables réelles à valeurs réelles supposée continûment différentiable, c'est-à-dire les ensembles de la forme :

$\{(x,y) \in \mathbb{R}^2$ $\vert$  $f(x,y) = c \}$ où $c \in \mathbb{R}$


In [1]:
from math import*
import numpy as np
import matplotlib.pyplot as plt
import autograd 
from autograd import numpy as np

### Contour Simple

On suppose dans un premier temps que la fontion $f$ est définie dans le __carré unité__ $[0,1]^2$ et on limite notre recherche aux lignes de niveaux qui possèdent un __point sur l'arête gauche du domaine de définition__ (de la forme $(0,y)$ pour $0 \le y \le 1$.) 

#### Amorce 

* On a $f$ une fonction continûment différentiable :

$$\begin{array}{ccccc}
f & : & [0,1]^2 & \to & \mathbb{R} \\
& & (x,y) & \mapsto & f(x,y) \\
\end{array}$$

* On définit $h$ la deuxième application partielle de $f$ au point $(0,0)$. 

$$\begin{array}{ccccc}
h & : & [0,1] & \to & \mathbb{R} \\
& & t & \mapsto & f(0,t) \\
\end{array}$$

$h$ est __continue__ car $f$ est continûment différentiable.

On cherche une condition raisonnable portant sur $f(0,0)$, $f(0,1)$ et $c \in \mathbb{R}$ telle qu'on soit certain qu'il existe un $t \in [0,1]$  tel que $f(0,t) = c$, soit telle que $h(t) = c$.

* Si $c \in [\min_{[0,1]}(h(t)),\max_{[0,1]}(h(t))]$, alors d'après le Théorème des Valeurs Intermédiaires, l'équation $h(t) =c$ pour $t \in [0,1]$ admet au minimum une solution.

* En particulier, si $c \in [\min(h(0),h(1)), \max(h(0),h(1)) \subset [\min_{[0,1]}(h(t)),\max_{[0,1]}(h(t))]$, alors d'après le TVI, l'équation $h(t) =c$ pour $t \in [0,1]$ admet au minimum une solution.


In [7]:
def f(x,y):
    return x**2+y**2+1

a=0

def h(t):
    return f(a,t)
    
def find_seed(h, c = 0.0, eps = 2**(-26)): 
    debut = 0
    fin = 1
    if (c<min(h(debut),h(fin))) or (c>max(h(debut),h(fin))): 
        print('bonjour')
        return None #c'est ici que ca bugge
    ecart = fin-debut
    while (ecart > eps):
        t = (debut+fin)/2
        if (h(t) < c):
            debut = t
        else:
            fin = t
        ecart = fin-debut
    return t 

find_seed(h, c = 1.5, eps = 2**(-26))

0.7071067839860916

#### Propagation

*Méthode :* 

On se place sur l'espace $[0,1]^2$. On a trouvé par dichotomie la **racine de la fonction** sur l'arête gauche gràce à la fonction `find_seed`. On calcule le **gradient** de la fonction, supposé non nul, en ce point puis son **orthogonale**, qui correspond à la **tangente** en ce point de la fonction. On se déplace d'un entier `delta` sur la tangente puis à l'aide de la méthode de Newton, on cherche le point le plus proche qui appartient à la courbe de niveau. 

*Mise en application :*

On cheche à implémenter une fonction qui renvoie un fragment de ligne de niveau de valeur $c$ de $f$ sous la forme de deux tableaux à une dimension d'abscisses et d'ordonnées de points de cette ligne, approximativement espacés de `delta`. Si un tel fragment ne peut être généré, la fonction renvoie deux tableaux vides.

In [11]:
def grad(f, x, y):#x et y sont les coordonnées d'un point
    g = autograd.grad 
    return np.r_[g(f,0)(x,y), g(f,1)(x,y)]
    
def perpendiculaire(v):
    return np.array([-v[1],v[0]])

def fin_ligne(x,y,delta):#ici x et y sont deux tableaux de coordonnées de point
    if (x[-1] > 1-delta) or (x[-1] < delta) or (y[-1] > 1-delta) or (y[-1] < delta):
        return False
    else:
        return True

def distance(x,y,a,b):
    return (sqrt((x-a)**2+(y-b)**2))



def newton(f,x,y,c,delta,eps=2**(-26)):#x et y sont les coordonnées du point précédent
    def F(a,b):
        return (f(a,b)-c,distance(x,y,a,b))
    def J_F(a,b):
        j=autograd.jacobian
        return np.c_[j(f,0)(a,b),j(f,1)(x,y)]
    a=x
    b=y
    while F(a,b)[0]>eps and F(a,b)[1]>delta:
        inverse_jacobienne=np.linlg(J_F(a,b))
        inter=(np.array([a,b])-np.dot(inverse_jacobienne,F(a,b)))[0]
        b=(np.array([a,b])-np.dot(inverse_jacobienne,F(a,b)))[1]
        a=inter
    return np.array([a,b])
    
def simple_contour_propagation(f, c = 0.0, delta=0.01):
    x1 = []
    y1 = []
    a = 0
    def h(t):
        return f(a,t)
    t = find_seed(h,c=1.5,eps=2**(-26))
    x1.append(0.0)
    y1.append(t)#on a ajouté le premier point du contour sur l'axe x=0
    gradient=grad(f,0.0,t)
    perp=perpendiculaire(gradient)*delta/np.linalg.norm(gradient)
    x1.append(perp[0]+x1[-1])
    y1.append(perp[1]+y1[-1])
    #là il faudrait faire le newton
    while fin_ligne(x1,y1,delta)==True:
        gradient=grad(f,float(x1[-1]),float(y1[-1]))
        perp=perpendiculaire(gradient)*delta/np.linalg.norm(gradient)
        perp2=-perp
        #on choisit le "bon" contour
        if distance(x,y,perp[0]+x1[-1],perp[0]+y1[-1]) < distance(x,y,perp2[0]+x1[-1],perp2[0]+y1[-1]):
            x1.append(perp2[0]+x1[-1])
            y1.append(perp2[1]+y1[-1])
        else:
            x1.append(perp[0]+x1[-1])
            y1.append(perp[1]+y1[-1])
        #il faudraitfaire le newton
    x=np.array(x1)
    y=np.array(y1)
    return x,y


def simple_contour(f, c = 0.0, delta=0.01):
    x1 = []
    y1 = []
    a = 0
    def h(t):
        return f(a,t)
    t = find_seed(h,c=0.3,eps=2**(-26))
    x1.append(0.0)#à modifier pour la fct contour
    y1.append(t)#on a ajouté le premier point du contour sur l'axe x=0
    gradient=grad(f,0.0,t)
    perp=perpendiculaire(gradient)*delta/np.linalg.norm(gradient)
    x1_inter=perp[0]+x1[-1]
    y1_inter=perp[1]+y1[-1]
    l=newton(f,x1_inter,y1_inter,2**(-26),delta,c)
    x1.append(l[0])
    y1.append(l[1])
    while fin_ligne(x1,y1,delta)==True:
        gradient=grad(f,float(x1[-1]),float(y1[-1]))
        perp=perpendiculaire(gradient)*delta/np.linalg.norm(gradient)
        perp2=-perp
        #on choisit le "bon" contour
        if distance(x,y,perp[0]+x1[-1],perp[0]+y1[-1]) < distance(x,y,perp2[0]+x1[-1],perp2[0]+y1[-1]):
            x1_inter=perp2[0]+x1[-1]
            y1_inter=perp2[1]+y1[-1]
            l=newton(f,x1_inter,y1_inter,2**(-26),delta,c)
            x1.append(l[0])
            y1.append(l[1])
        else:
            x1_inter=perp[0]+x1[-1]
            y1_inter=perp[1]+y1[-1]
            l=newton(f,x1_inter,y1_inter,2**(-26),delta,c)
            x1.append(l[0])
            y1.append(l[1])
    x=np.array(x1)
    y=np.array(y1)
    return x,y



def f(x,y):
    return float(x)**2+float(y)**2

x,y = simple_contour_propagation(f, c=0.3, delta = 0.01)

plt.plot(x,y)


bonjour


TypeError: float() argument must be a string or a number, not 'ArrayBox'